# TEVA Output Explorer Dashboard
The intent of this dashboard is to facilitate exploration of the TEVA algorithm output by providing several ways to view and interact with the results.

### The dashboard has several primary components:
- Positive Predictive Value (PPV) vs. Coverage (COV) plot
- Adjustable sensitivity range
- Feature distributions and value ranges for selected CC
- Confusion matrix for selected CC
- CC Feature Pairing
- Features used by CCs
- CCs used by DNFs

## Import Packages

In [16]:
# Base packages
import numpy as np
import pandas as pd
from matplotlib.pyplot import get_cmap
from matplotlib.colors import rgb2hex

# Holoviews packages
import holoviews as hv
import hvplot.pandas
from bokeh.io import output_notebook
from bokeh.palettes import varying_alpha_palette, Category20
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar, TabPanel, Tabs
import panel as pn
import colorcet as cc
output_notebook()
hv.extension('bokeh')
pn.extension(throttled=True)

# Custom post processing and plotting functions
import TEVA_Post_Processing as post
import TEVA_Dynamic_Plotting as teva_plot

Loading BokehJS ...

## Import TEVA Output Files

In [17]:
# # Import CC and DNF output files
# ccs = pd.read_excel('Sample_Data/ccs_2DOC_CAMELS_1_S_True_60_60_TEVA007.xlsx', sheet_name='CCEA_Low')
# dnfs = pd.read_excel('Sample_Data/dnfs_2DOC_CAMELS_1_S_True_60_60_TEVA007.xlsx', sheet_name='DNFEA_Low')
# # Import observation data
# data = pd.read_csv('Sample_Data/test_observations.csv')




# CRB
ccs = pd.read_excel('/Users/ryanvan/Library/CloudStorage/OneDrive-UniversityofVermont/Documents/_UVM/Research/CIROH/_CRB Study/CRB-Study-Code/TEVA/NWM_v3_TEVA_test_CC_sens_True.xlsx', sheet_name='CCEA_3')
dnfs = pd.read_excel('/Users/ryanvan/Library/CloudStorage/OneDrive-UniversityofVermont/Documents/_UVM/Research/CIROH/_CRB Study/CRB-Study-Code/TEVA/NWM_v3_TEVA_test_DNF_sens_True.xlsx', sheet_name='DNFEA_3')
data = pd.read_csv('/Users/ryanvan/Library/CloudStorage/OneDrive-UniversityofVermont/Documents/_UVM/Research/CIROH/_CRB Study/Baseflow Data/BFI_GAGES_recoded.csv')
data.drop(columns=['usgs_ID', 'Cluster 4'], inplace=True)





# # # Import CC and DNF output files
# ccs = pd.read_excel('/Users/ryanvan/Library/CloudStorage/OneDrive-UniversityofVermont/Documents/_UVM/Research/CIROH/TEVA/BFI Data/BFI_CC_sensitivity_true.xlsx', sheet_name='CCEA_1')
# dnfs = pd.read_excel('/Users/ryanvan/Library/CloudStorage/OneDrive-UniversityofVermont/Documents/_UVM/Research/CIROH/TEVA/BFI Data/BFI_DNF_sensitivity_true.xlsx', sheet_name='DNFEA_1')
# # Import observation data
# data = pd.read_csv('/Users/ryanvan/Library/CloudStorage/OneDrive-UniversityofVermont/Documents/_UVM/Research/CIROH/TEVA/BFI Data/observation_data.csv')





# Import CC and DNF output files
# ccs = pd.read_excel('/Users/ryanvan/Library/CloudStorage/OneDrive-UniversityofVermont/Documents/_UVM/Research/CIROH/NWM/d-score Metrics/NWM_v3_TEVA_test_CC_sens_False.xlsx', sheet_name='CCEA_5')
# dnfs = pd.read_excel('/Users/ryanvan/Library/CloudStorage/OneDrive-UniversityofVermont/Documents/_UVM/Research/CIROH/NWM/d-score Metrics/NWM_v3_TEVA_test_DNF_sens_False.xlsx', sheet_name='DNFEA_5')

# # Import observation data
# data = pd.read_csv('/Users/ryanvan/Library/CloudStorage/OneDrive-UniversityofVermont/Documents/_UVM/Research/CIROH/NWM/d-score Metrics/NWM_v3_MSE_Decomp_TEVA_test_observations.csv')

## Run Post-Processing Functions
These functions, imported from the "TEVA_Post_Processing.py" file, help parse the Excel spreadsheets into a more usable form to be used by the plotting functions.

In [18]:
# List of the CCs composing each DNF
all_ccs = post.parse_dnf(dnfs)
all_ccs_flat = post.flatten(all_ccs)

# List of the features composing each CC
cc_features = post.parse_cc(ccs)
unique_features = pd.unique(post.flatten(cc_features))
all_features_flat = post.flatten(cc_features)

# List of the unique CCs across all DNFs
unique_ccs = (np.unique(all_ccs_flat))

# Fitness contours
x_fit, y_fit, z_fit, fitness = post.fitness_contours(1000, dnfs, ccs)

# CC feature usage image matrix
cc_matrix = post.CC_feature_heatmap(unique_features, cc_features)

# CC and DNF lengths (needed later for plotting)
cc_len = np.arange(1, max(ccs['order']) + 1, 1)
dnf_len = np.arange(1, max(dnfs['order']) + 1, 1)

# Stacked feature and cc dataframes for bar chart
stacked_features, stacked_feature_names = post.stacked_features(ccs, unique_features, cc_features, all_features_flat)
stacked_ccs, stacked_cc_names = post.stacked_ccs(dnfs, unique_ccs, all_ccs, all_ccs_flat)

# feature value ranges by cc
feature_values_by_cc = post.feature_ranges_by_cc(ccs)

## Dashboard Support
- Custom color maps for cc/dnf plot, fitness contours, and cc pairing heatmap.
- Data structures used for bokeh plotting.

In [19]:
# Custom color maps
'''
Create custom colormaps, one for CCs, one from DNFs, and one for fitness contours.
Colormaps can range from 0 to 256, but it is best to trim the lightest and darkest portions out.
'''
cc_colors = []
dnf_colors = []

# CCs and DNFs
for i in range(20,220):
    cc_colors.append(rgb2hex(get_cmap('Blues_r')(i)))
    dnf_colors.append(rgb2hex(get_cmap('Oranges_r')(i)))

# Fitness contours
contour_colors = varying_alpha_palette(color='black', start_alpha=150, end_alpha=10)

# For CC feature heatmap
cc_heatmap_colormap = []
for i in range(0,256):
    cc_heatmap_colormap.append(rgb2hex(get_cmap('Blues')(i)))

In [20]:
# Bokeh Data Sources
'''
Bokeh uses a data structure called a "Column Data Source" (CDS) for plotting.
The easiest way to create them with your data is by passing your data as a disctionary.
'''

# column data source for CCs
cc_plot_data = {'x_values': ccs['cov'],
                'y_values': ccs['ppv'],
                'min_sens': ccs['min_feat_sensitivity'],
                'max_sens': ccs['max_feat_sensitivity'],
                'CC': ccs['Unnamed: 0'],
                'Order': ccs['order'],
                'Features': cc_features}
cc_plot_source = ColumnDataSource(data=cc_plot_data)
cc_plot_data= pd.DataFrame(cc_plot_data)

# column data source for DNFs
dnf_plot_data = {'x_values': dnfs['cov'],
                 'y_values': dnfs['ppv'],
                 'Order': dnfs['order'],
                 'DNF': dnfs['Unnamed: 0'],
                 'CCs': all_ccs}
dnf_plot_source = ColumnDataSource(data=dnf_plot_data)

# column data source for fitness contours
dnf_cont_data = {'x_values': x_fit,
                 'y_values': y_fit,
                 'z_values': z_fit}
dnf_cont_source = ColumnDataSource(dnf_cont_data)

# CC feature image
cc_image_data = {
    'image': [cc_matrix],
    'x': [0],
    'y': [0],
    'dw': [len(unique_features)],
    'dh': [len(unique_features)]
}

cc_image_data = ColumnDataSource(data=cc_image_data)

## Static Plots
- CC feature pairing
- CC feature usage
- DNF CC usage

In [21]:
# Bokeh cc feature heatmap setup
# Hover tooltip
cc_image_hover = [
    ('Count', '@image')
]

# Figure setup
cc_heatmap = figure(height=400,
                    aspect_ratio=1,
                    x_range=unique_features,
                    y_range=unique_features,    
                    match_aspect=True,
                    tools=['hover', 'crosshair', 'save'],
                    tooltips=cc_image_hover)

# Color map for data
img_color_mapper = LinearColorMapper(palette=cc_heatmap_colormap, low=0, high=len(unique_features))
# Imshow
img = cc_heatmap.image(source=cc_image_data, x='x', y='y', dw='dw', dh='dh', color_mapper=img_color_mapper)
# Create colorbar
color_bar = ColorBar(color_mapper=img_color_mapper, label_standoff=12, major_tick_line_color='black')
# Add colorbar to figure
cc_heatmap.add_layout(color_bar, 'right')

# color for nan values
img.glyph.color_mapper.nan_color = (0, 0, 0, 0)

# rotate x labels
cc_heatmap.xaxis.major_label_orientation = np.pi/2

# turn off grid
cc_heatmap.xgrid.visible = False
cc_heatmap.ygrid.visible = False

# Add to tab
tab1 = TabPanel(child=cc_heatmap, title='Feature Pairing')

In [22]:
#### FIGURE 2 CC Feature Usage
# categorical colormap
cat_map = cc.palette['glasbey_bw']
p2 = figure(width = 800, height = 500,
            x_range=stacked_features['Feature'],
            x_axis_label='Feature',
            y_axis_label='Count',
            hidpi=True,
            tools='crosshair, reset, save, help')

p2.vbar_stack(stacked_feature_names,
              x='Feature',
              width=0.6,
              color=cat_map[0:len(cc_len)],
              source=stacked_features,
              legend_label=stacked_feature_names)

# General formatting
p2.xaxis.major_label_orientation = 1
p2.y_range.start = 0
p2.legend.location = 'top_right'
p2.legend.orientation = 'vertical'

tab2 = TabPanel(child=p2, title='CC: Feature Usage')

#### FIGURE 3 dnf cc usage

p3 = figure(width = 800, height = 500,
            x_range=stacked_ccs['CC'],
            x_axis_label='CC',
            y_axis_label='Count',
            hidpi=True,
            tools='crosshair, reset, save, help')

p3.vbar_stack(stacked_cc_names,
              x='CC',
              width=0.6,
              color=cat_map[0:len(dnf_len)],
              source=stacked_ccs,
              legend_label=stacked_cc_names)

# General formatting
p3.xaxis.major_label_orientation = 1
p3.y_range.start = 0
p3.legend.location = 'top_right'
p3.legend.orientation = 'vertical'

tab3 = TabPanel(child=p3, title='DNF: CC Usage')

## Dashboard Interactivity
- Create interactive widgets.
- Link widgets to dynamic plots.

In [23]:
# WIDGETS
# Dropdown for selecting CC to plot
dropdown_options = list(np.sort(unique_ccs.astype(int)))
cc_select = pn.widgets.Select(options=dropdown_options, width=75, name='CC Select', description='Select a CC to view features.')

# Slider for sensitivity
sens_slider_max = pn.widgets.IntSlider(name = 'Max. Feature Sensitivity, 10^',
                                       start = int(np.ceil(min(ccs['max_feat_sensitivity']))),
                                       end = 0,
                                       step = 1,
                                       value = 0,
                                       bar_color='#ffffff'
                                       )

# create list of min options
if len(np.floor(ccs['min_feat_sensitivity'][np.isinf(ccs['min_feat_sensitivity'])==False])) == 0:
    slider_options = [-np.inf]
else:
    slider_options = [-np.inf]
    for i in range(int(min(np.floor(ccs['min_feat_sensitivity'][np.isinf(ccs['min_feat_sensitivity'])==False]))), 1):
        slider_options.append(int(i))

sens_slider_min = pn.widgets.DiscreteSlider(name = 'Min. Feature Sensitivity, 10^',
                                            options = slider_options,
                                            value = -np.inf
                                            )
# BINDS

# Bind function to widget
dynamic_subplots = pn.bind(teva_plot.feature_plotter, selected_cc=cc_select, data=data, cc_features=cc_features, feature_values_by_cc=feature_values_by_cc)
dynamic_confusion_matrix = pn.bind(teva_plot.confusion_matrix_plotter, selected_cc=cc_select, ccs=ccs)
dynamic_cc = pn.bind(teva_plot.cc_plotter, min_sens=sens_slider_min, max_sens=sens_slider_max, fitness=fitness, x_fit=x_fit, y_fit=y_fit, z_fit=z_fit, contour_colors=contour_colors, cc_plot_data=cc_plot_data, cc_len=cc_len, cc_plot_source=cc_plot_source, cc_colors=cc_colors, ccs=ccs, dnf_len=dnf_len, dnf_plot_data=dnf_plot_data, dnf_plot_source=dnf_plot_source, dnf_colors=dnf_colors, dnfs=dnfs)

## Dashboard Layout
- Layout the dashboard elements
- Launch the dashboard in a web browser using local server

In [24]:
controls_style = {
    'background': 'cornflowerblue',
    'border': '1px solid black',
    'padding': '10px'
}


app = pn.Column(
    '# TEVA Output Explorer',
    pn.Row('## Controls', sens_slider_min, sens_slider_max, cc_select, height=70, width=1000, styles=controls_style, width_policy='max'),
    pn.Row(dynamic_cc, dynamic_subplots),
    pn.Row(Tabs(tabs=[tab1, tab2, tab3]), dynamic_confusion_matrix))

print(app)

Column
    [0] Markdown(str)
    [1] Row(height=70, styles={'background': 'cornflower...}, width=1000, width_policy='max')
        [0] Markdown(str)
        [1] DiscreteSlider(name='Min. Feature Sensitivity,..., options=[-inf, -4, -3, -2, -1, 0], value=-inf)
        [2] IntSlider(bar_color='#ffffff', end=0, name='Max. Feature Sensitivity,..., start=-2)
        [3] Select(description='Select a CC t..., name='CC Select', options=[np.int64(0), ...], value=np.int64(0), width=75)
    [2] Row
        [0] ParamFunction(function, _pane=Bokeh, defer_load=False)
        [1] ParamFunction(function, _pane=FlexBox, defer_load=False)
    [3] Row
        [0] Bokeh(Tabs)
        [1] ParamFunction(function, _pane=Bokeh, defer_load=False)


In [25]:
# Run in browser
app.show()

Launching server at http://localhost:54405
